Parts of the code taken from the original paper on DAAM. Source: https://github.com/castorini/daam/blob/main/notebooks/1-visuosyntactic-analyses.ipynb

#Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Go to the DAAM folder in Google Drive
folder_path = '/content/drive/My Drive/Colab Notebooks/Stable diffusion/DAAM Wrong'
os.chdir(folder_path)

# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/drive/My Drive/Colab Notebooks/Stable diffusion/DAAM Wrong


In [4]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 3

In [5]:
import torch

torch.cuda.amp.autocast().__enter__()
torch.set_grad_enabled(False);

In [6]:
import daam

In [7]:
from diffusers import StableDiffusionPipeline

In [8]:
from daam import set_seed, trace
pipe = StableDiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-2-1-base')
pipe.to('cuda:0');

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
import pandas as pd
df = pd.read_csv('final_coco_dataframe.csv')

In [10]:
# Drop the columns "preposition" and "caption"
df = df.drop(columns=['preposition', 'caption'])

# Rename the columns "wrong_preposition" to "preposition" and "wrong_caption" to "caption"
df = df.rename(columns={'wrong_preposition': 'preposition', 'wrong_caption': 'caption'})

# Take only the first 450 rows of the DataFrame
df = df.head(450)

# Print the head of the DataFrame to check the changes
print(df.head())

   image_id      id       subject          object     verb preposition  \
0    321333  770625     A picture  two young kids   posing       above   
1    366611  765550           dog     soccer ball   laying      out of   
2    490171  370135  a surf board       the water   riding      within   
3    491757  694991         A cat       blue eyes  sitting     outside   
4    247806  768641       A clock    a bell tower      NaN     towards   

                                             caption  
0  A picture above two young kids posing or a pic...  
1      a dog laying out of a soccer ball on a field.  
2  A man and a dog riding a surf board within the...  
3     A cat outside blue eyes sitting on a pink bed.  
4     A clock towards a bell tower of an old church.  


#Generate DAAM Maps

In [11]:
torch.cuda.empty_cache()

In [12]:
import torch
from tqdm import tqdm
from pathlib import Path

def generate_images(df, start_idx=0, batch_size=2):
    total_batches = (len(df) - start_idx) // batch_size + ((len(df) - start_idx) % batch_size > 0)
    output_folder = Path('experiments')
    output_folder.mkdir(exist_ok=True)

    for batch_num in range(total_batches):
        start = start_idx + batch_num * batch_size
        end = min(start + batch_size, len(df))
        batch_df = df.iloc[start:end]

        for _, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f'Batch {batch_num+1}/{total_batches}'):
            id, caption = row['id'], row['caption']
            try:
                gen = set_seed(id)
                with torch.no_grad():
                    with trace(pipe) as tc:
                        out = pipe(caption, num_inference_steps=20, generator=gen)
                        exp = tc.to_experiment(output_folder, id=str(id), seed=id)
                        exp.save(output_folder, heat_maps=False)
                torch.cuda.empty_cache()
            except Exception as e:
                print(f"Error processing id {id}: {e}")

        # Save progress after each batch
        with open('progress.txt', 'w') as f:
            f.write(str(end))

# Check for existing progress and resume
try:
    with open('progress.txt', 'r') as f:
        start_index = int(f.read().strip())
except FileNotFoundError:
    start_index = 0

generate_images(df, start_idx=start_index)

Batch 1/225:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Batch 1/225:  50%|█████     | 1/2 [00:06<00:06,  6.50s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

Batch 2/225:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batch 225/225:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Batch 225/225:  50%|█████     | 1/2 [00:04<00:04,  4.26s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

Batch 225/225: 100%|██████████| 2/2 [00:08<00:00,  4.41s/it]


In [13]:
# Set the path to the 'experiments' directory
experiment_path = Path('experiments')

# Use a generator expression to count directories
folder_count = sum(1 for entry in experiment_path.iterdir() if entry.is_dir())

print(f"There are {folder_count} folders in {experiment_path}")

There are 450 folders in experiments


# Parse and analyse

In [14]:
from matplotlib import pyplot as plt
from daam import GenerationExperiment

In [15]:
def iou(a, b, t: float = 0.1) -> float:
    intersection = (a > t) & (b > t)
    union = (a > t) | (b > t)

    i = intersection.float().sum()
    u = union.float().sum()

    if u < 1e-6:
        return 0.0
    else:
        return (i / u).item()

In [16]:
import pandas as pd
import gc
from pathlib import Path
from tqdm import tqdm
from daam import GenerationExperiment

print("DataFrame loaded with {} rows.".format(len(df)))

stats = []

# Use itertuples for more efficient row iteration
for row in tqdm(df.itertuples(index=True), total=len(df)):
    experiment_path = Path('experiments') / str(row.id)
    if experiment_path.exists():
        print(f"Loading experiment from: {experiment_path}")
        exp = GenerationExperiment.load(experiment_path)
    else:
        print(f"Experiment path does not exist: {experiment_path}")
        continue

    try:
        # Assuming heat_map is a method that can be called without much overhead
        heat_map = exp.heat_map()
        word_maps = {}
        words_of_interest = ['preposition', 'subject', 'object', 'verb']

        for word_type in words_of_interest:
            word = getattr(row, word_type)
            if pd.notna(word):
                try:
                    word_maps[word_type] = heat_map.compute_word_heat_map(word).value.cuda()
                except ValueError as ve:
                    print(f"Could not compute heat map for {word_type}: {word}, Error: {ve}")

        pairs_of_interest = [('preposition', 'subject'), ('preposition', 'object'), ('preposition', 'verb')]
        for head_type, dep_type in pairs_of_interest:
            if head_type in word_maps and dep_type in word_maps:
                iou_value = iou(word_maps[head_type], word_maps[dep_type])
                stats.append({
                    'pair': f"{head_type}-{dep_type}",
                    'preposition': getattr(row, 'preposition'),
                    'iou': iou_value,
                    'experiment_path': str(experiment_path)
                })

    except Exception as e:
        print(f"Error processing {str(experiment_path)}: {str(e)}")

    # Clear memory after each row is processed
    del exp
    gc.collect()

stats_df = pd.DataFrame(stats)
print("Statistics collected:")
print(stats_df)

DataFrame loaded with 450 rows.


  0%|          | 0/450 [00:00<?, ?it/s]

Loading experiment from: experiments/770625


  0%|          | 1/450 [00:00<04:13,  1.77it/s]

Loading experiment from: experiments/765550


...

Loading experiment from: experiments/576134


100%|█████████▉| 449/450 [03:46<00:00,  2.11it/s]

Loading experiment from: experiments/151599


100%|██████████| 450/450 [03:46<00:00,  1.99it/s]

Statistics collected:
                     pair preposition       iou     experiment_path
0     preposition-subject       above  0.000000  experiments/770625
1      preposition-object       above  0.079439  experiments/770625
2        preposition-verb       above  0.002070  experiments/770625
3     preposition-subject      out of  0.000000  experiments/765550
4      preposition-object      out of  0.000000  experiments/765550
...                   ...         ...       ...                 ...
1096  preposition-subject       up at  0.000000  experiments/576134
1097   preposition-object       up at  0.000000  experiments/576134
1098  preposition-subject       up to  0.000000  experiments/151599
1099   preposition-object       up to  0.000000  experiments/151599
1100     preposition-verb       up to  0.000000  experiments/151599

[1101 rows x 4 columns]


#Aggregate statistics

In [17]:
stats_df['preposition'] = stats_df['preposition'].str.lower()

In [18]:
stats_df.to_csv('stats_output.csv', index=False)

In [19]:
preposition_counts = stats_df['preposition'].value_counts()

# Convert the Series to a DataFrame for better tabular presentation
preposition_counts_df = preposition_counts.reset_index()
preposition_counts_df.columns = ['Preposition', 'Count']

# Print the resulting DataFrame
print(preposition_counts_df)

    Preposition  Count
0         above     33
1       outside     33
2       against     32
3            at     31
4            of     30
..          ...    ...
56      wearing      8
57      back to      8
58    inside of      8
59  in front of      6
60      through      4

[61 rows x 2 columns]


threshold 0.1

In [20]:
# Calculate the mean IoU across all pairs, ignoring prepositions
mean_iou_all = stats_df['iou'].mean()
rounded_mean_iou_percentage = round(mean_iou_all * 100, 4)
print("Mean IoU across all pairs:", rounded_mean_iou_percentage, "%")

Mean IoU across all pairs: 3.9373 %


In [21]:
# Calculate the mean IoU for each type of pair, ignoring prepositions
mean_iou_by_pair = stats_df.groupby('pair').agg(mean_iou=('iou', 'mean'))

# Convert to percentage and round to two decimal places
mean_iou_by_pair['mean_iou'] = (mean_iou_by_pair['mean_iou'] * 100).round(4)

# Sort from largest to smallest mean IoU
mean_iou_by_pair = mean_iou_by_pair.sort_values('mean_iou', ascending=False)

# Print the formatted DataFrame
print("Mean IoU by Pair (in %):")
print(mean_iou_by_pair)

Mean IoU by Pair (in %):
                     mean_iou
pair                         
preposition-subject    6.4334
preposition-verb       2.8330
preposition-object     2.2512


In [22]:
import pandas as pd

# Adjust display settings to show full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Calculate the mean IoU and count for each preposition
mean_iou_by_preposition = stats_df.groupby('preposition').agg(
    mean_iou=('iou', 'mean'),
    count=('iou', 'count')  # Add count of occurrences for each preposition
)

# Convert mean IoU to percentage and round to four decimal places
mean_iou_by_preposition['mean_iou'] = (mean_iou_by_preposition['mean_iou'] * 100).round(4)

# Sort from largest to smallest by mean IoU
mean_iou_by_preposition = mean_iou_by_preposition.sort_values('mean_iou', ascending=False)

# Display only the first 10 entries
mean_iou_by_preposition = mean_iou_by_preposition.head(30)

# Print the formatted DataFrame
print("Mean IoU by Preposition (in %), including counts:")
print(mean_iou_by_preposition)

Mean IoU by Preposition (in %), including counts:
             mean_iou  count
preposition                 
around        17.3033     15
towards       14.6391     17
across        14.4956     22
during        12.7049     15
down          11.7368     18
with          10.4236     23
between       10.3090     16
underneath     9.7202      9
over           9.0317     18
at             8.0500     31
on             6.9435     24
behind         6.5046     11
against        6.3417     32
into           6.2519     20
outside        5.7913     33
among          5.5326     27
in to          5.1457     24
down into      5.0200     15
in             4.7577     17
under          4.6821     22
about          4.4161     17
beside         4.3361     16
off            3.7499     24
in and out     3.5403     15
beyond         3.0795     10
as             2.7614     11
while          2.7308     18
onto           2.7251     19
near           2.6538     20
covered        2.5551     15


In [23]:
import pandas as pd

# Adjust display settings to show full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Calculate the mean IoU and count for each type of pair and each preposition
mean_iou_by_pair_and_preposition = stats_df.groupby(['pair', 'preposition']).agg(
    mean_iou=('iou', 'mean'),
    preposition_count=('iou', 'count')  # Count occurrences of each preposition within each pair
)

# Convert mean IoU to percentage and round to four decimal places
mean_iou_by_pair_and_preposition['mean_iou'] = (mean_iou_by_pair_and_preposition['mean_iou'] * 100).round(4)

# Sort within each 'pair' group from largest to smallest by mean IoU
mean_iou_by_pair_and_preposition = mean_iou_by_pair_and_preposition.sort_values(['pair', 'mean_iou'], ascending=[True, False])

# Use groupby on 'pair' and apply a lambda to take the top 10 prepositions for each pair
# Reset the index properly to keep the 'pair' and 'preposition' columns
top_10_per_pair = mean_iou_by_pair_and_preposition.groupby('pair', group_keys=False).apply(lambda x: x.head(10)).reset_index()

# Print the formatted DataFrame
print("Mean IoU by Pair and Preposition (in %), including preposition counts:")
print(top_10_per_pair)

Mean IoU by Pair and Preposition (in %), including preposition counts:
                   pair preposition  mean_iou  preposition_count
0    preposition-object     towards   17.2047                  7
1    preposition-object        down   11.2391                  6
2    preposition-object     against    8.6810                 12
3    preposition-object       while    7.0220                  7
4    preposition-object      around    6.2255                  5
5    preposition-object         off    5.7830                  9
6    preposition-object        into    5.4331                  7
7    preposition-object      during    5.3276                  6
8    preposition-object       under    4.6199                  8
9    preposition-object      across    4.5877                  9
10  preposition-subject        with   27.2618                  8
11  preposition-subject      during   26.4347                  6
12  preposition-subject        down   23.4674                  6
13  preposition-sub